In [1]:
class CircularTapeTM:                               # Define a nova classe da máquina de Turing circular
    """
    Máquina de Turing com fita circular de tamanho fixo.          # Docstring: descreve o que é a classe
    - tape: string inicial (o comprimento define o tamanho do anel).
    - transitions: dict {(estado, símbolo_lido): (símbolo_escrito, dir, prox_estado)}
      dir ∈ {'R', 'L', 'N'} (N = não mover).
    """

    def __init__(self, tape, transitions, start, accept, reject):
        # ---------- atributos básicos ----------
        self.tape  = list(tape)                   # Converte a string da fita para lista mutável de símbolos
        self.n     = len(self.tape)               # Número fixo de células (tamanho do “anel”)
        self.tr    = transitions                  # Dicionário de transições (tabela δ)
        self.state = start                        # Estado atual da máquina
        self.head  = 0                            # Posição inicial do cabeçote (célula 0)
        self.ACC   = accept                       # Estado de aceitação
        self.REJ   = reject                       # Estado de rejeição

    # ---------- função auxiliar para mover o cabeçote ----------
    def _move(self, direction):
        if direction == 'R':                      # Se a regra diz “R” (right)…
            self.head = (self.head + 1) % self.n  # Avança 1 célula; “% n” faz o loop circular
        elif direction == 'L':                    # Se a regra diz “L” (left)…
            self.head = (self.head - 1) % self.n  # Retrocede 1 célula; também circular
        # Se direction == 'N', não faz nada (não move)

    # ---------- executa 1 passo da computação ----------
    def step(self):
        sym = self.tape[self.head]               # Lê o símbolo sob o cabeçote
        key = (self.state, sym)                  # Monta a chave (estado, símbolo) p/ buscar na tabela

        if key not in self.tr:                   # Se não existe transição definida
            self.state = self.REJ                # Vai direto para o estado de rejeição
            return                               # Termina o passo

        # Caso haja transição:
        new_sym, dir_, nxt = self.tr[key]        # Desempacota: símbolo a escrever, direção, próximo estado
        self.tape[self.head] = new_sym           # Escreve o novo símbolo na fita
        self.state = nxt                         # Atualiza o estado
        self._move(dir_)                         # Move o cabeçote conforme a direção

    # ---------- executa vários passos até aceitar/rejeitar ou atingir limite ----------
    def run(self, max_steps=10_000):
        steps = 0                                # Contador de passos (serve p/ debug ou evitar loop infinito)
        while self.state not in {self.ACC, self.REJ} and steps < max_steps:
            self.step()                          # Executa um passo
            steps += 1                           # Incrementa o contador

        # Retorna tupla: (True se aceitou, fita final como string, nº de passos usados)
        return (self.state == self.ACC), ''.join(self.tape), steps


Paridade de 1s em anel de 4 células

In [ ]:
tr_paridade = {
    ('q0', '1'): ('X', 'R', 'q1'),   # Marca 1º '1'
    ('q0', 'X'): ('X', 'R', 'q0'),

    ('q1', '1'): ('1', 'R', 'q0'),   # Achou par
    ('q1', 'X'): ('X', 'R', 'q1'),

    # Se der uma volta e nenhum par foi achado → rejeita
    ('q0', 'X_end'): ('X_end', 'N', 'q_rej'),
    ('q1', 'X_end'): ('X_end', 'N', 'q_rej'),
}

# Inserimos símbolo especial 'X_end' em uma das posições para detectar a volta completa
fita = list("1111")
fita[-1] = 'X_end'            # marca fim lógico do ciclo

tm = CircularTapeTM(''.join(fita), tr_paridade, 'q0', 'q_acc', 'q_rej')
aceita, fita_final, passos = tm.run()
print("Aceita?", aceita)
print("Fita:", fita_final, "| passos:", passos)

Aceita? False
Fita: X1XX_end | passos: 5


Rotação circular à direita

In [ ]:
tr_rot = {
    # Estado q0: guarda símbolo da célula e avança
    ('q0', 'a'): ('_', 'R', 'q1'),  # '_' é marcador vazio
    ('q1', 'b'): ('a', 'R', 'q2'),
    ('q2', 'c'): ('b', 'R', 'q3'),
    ('q3', 'd'): ('c', 'R', 'q4'),
    ('q4', '_'): ('d', 'N', 'q_acc'),  # escreve último e termina
}

tm2 = CircularTapeTM('abcd', tr_rot, 'q0', 'q_acc', 'q_rej')
ok, out, _ = tm2.run()
print(out)        # saída esperada: "dabc"

dabc


Verificar paridade par de 1 s em anel de 6 células

In [ ]:
tr_par = {
    ('q0', '1'): ('X', 'R', 'q1'),   # marca 1º ‘1’
    ('q0', 'X'): ('X', 'R', 'q0'),

    ('q1', '1'): ('X', 'R', 'q0'),   # achou par → volta ao estado “par”
    ('q1', 'X'): ('X', 'R', 'q1'),

    ('q0', '_'): ('_', 'N', 'q_acc'),  # sem ‘1’ pendente → par → aceita
    ('q1', '_'): ('_', 'N', 'q_rej'),  # terminou com ‘1’ pendente → ímpar
}

tm1 = CircularTapeTM('111111', tr_par, 'q0', 'q_acc', 'q_rej')
print(tm1.run())  # (True, 'XXXXXX', passos)

(False, 'XXXXXX', 10000)


Rotação circular à direita (já mostrado, agora genérico)

In [ ]:
tr_rot = {
    ('q0', '_'): ('_', 'N', 'q_acc'),          # fita vazia → já concluiu
    ('q0', 'a'): ('_', 'R', 'q1'),             # guarda 1º símbolo
    ('q1', 'b'): ('a', 'R', 'q2'),
    ('q2', 'c'): ('b', 'R', 'q3'),
    ('q3', 'd'): ('c', 'R', 'q4'),
    ('q4', '_'): ('d', 'N', 'q_acc'),
}
tm2 = CircularTapeTM('abcd', tr_rot, 'q0', 'q_acc', 'q_rej')
print(tm2.run())  # ('dabc')

(True, 'dabc', 5)


Relógio binário mod 8

In [6]:
tr_clk = {
    # Estado q0: incrementa o bit menos significativo (posição head 0)
    ('q0', '0'): ('1', 'N', 'q_acc'),           # 0→1 e termina
    ('q0', '1'): ('0', 'L', 'q1'),              # 1→0 e vai para o bit seguinte

    # Estado q1: bit 1
    ('q1', '0'): ('1', 'N', 'q_acc'),
    ('q1', '1'): ('0', 'L', 'q2'),

    # Estado q2: bit 2
    ('q2', '0'): ('1', 'N', 'q_acc'),
    ('q2', '1'): ('0', 'N', 'q_acc'),           # overflow volta a 000
}

tm3 = CircularTapeTM('000', tr_clk, 'q0', 'q_acc', 'q_rej')
for i in range(10):
    ok, tape, _ = tm3.run()
    print(tape)     # imprime sequência binária
    tm3.state = 'q0'  # reseta estado para novo incremento

100
001
010
100
001
010
100
001
010
100


Alternar todos os bits (NOT) e parar após volta completa

In [ ]:
tape4 = list('010011')
tape4[0] = '#'  # marcador

tm4 = CircularTapeTM(''.join(tape4), {
    ('q0', '#'): ('#', 'R', 'q1'),      # estado q0 → só pula o marcador
    ('q1', '0'): ('1', 'R', 'q1'),      # troca 0 por 1
    ('q1', '1'): ('0', 'R', 'q1'),      # troca 1 por 0
    ('q1', '#'): ('#', 'N', 'q_acc'),   # quando der a volta → aceita
}, 'q0', 'q_acc', 'q_rej')

print(tm4.run())  #(True, '#101100', X)


(True, '#01100', 7)


Procurar símbolo especial e “acertar” ponteiro

In [8]:
tr_seek = {
    ('q0', '@'): ('@', 'N', 'q_acc'),
    ('q0', '0'): ('0', 'R', 'q0'),
    ('q0', '1'): ('1', 'R', 'q0'),
}

tm5 = CircularTapeTM('101@001', tr_seek, 'q0', 'q_acc', 'q_rej')
print(tm5.run())  # Aceita em poucos passos, fita intacta

(True, '101@001', 4)
